# scipp-widgets concepts


## Inputs
The input class describes how an input widget should be constructed and how its input should be processed. There are a number of generic classes provided. If you wish to make your own however the interface that needs to be impliment is one property and two methods.These are:
 * The `name` property, this should return a string and corrospond to the name of the underyling function input this input corrosponds to.
 * The `validator` method. This takes a single input and returns a single input. This method is called on the input value as a pre-processing step before it it passed to the underlying function. This function should do any neccecary pre-processing and throw a ValueError if the validation fails. See the validator section below for more details.
 * The `create_input_widget` method. This method should take no inputs and return the widget object you wish to use as the grpahical input.
 
 For example the base `Input` class looks like

In [ ]:
import ipywidgets as widgets
class Input():
    def __init__(self,
                 name,
                 validator=lambda input: input,
                 options=(),
                 tooltip=''):
        self.name = name
        self.validator = validator
        self._options = options
        self._tooltip = tooltip if tooltip else name

    def create_input_widget(self):
        return widgets.Combobox(placeholder=self._tooltip,
                                continuous_update=False,
                                options=self._options)

## Validators
A validator can be assigned to each input. These serve the dual purpose of doing ay required pre-processing and validating the user input. They should take the form of a function with a single input and single output and throw a ValueError with an approprate message if the validation fails. For example the `ValueValidator` class which validates that the user input is one of a set of allowed values looks like.

In [ ]:
class ValueValidator():
    def __init__(self, allowed_values):
        self.allowed_values = allowed_values

    def __call__(self, input):
        if input not in self.allowed_values:
            raise ValueError(
                f'{input} is invalid. Allowed values are{self.allowed_values}')
        return input

Note however that in principal only a function is required.